In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()

import pandas as pd
import numpy as np
import math

- Read the case, department, and source data into their own spark dataframes.
- Let's see how writing to the local disk works in spark:
- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
- Inspect your folder structure. What do you notice?
- Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [2]:
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)
source.show(1, vertical=True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row



In [3]:
case = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [4]:
dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)
dept.show(1, vertical=True)

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row



In [5]:
source.write.json("data/source_json", mode="overwrite")

In [6]:
source.printSchema(), case.printSchema(), dept.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



(None, None, None)

1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [7]:
case.show(5)

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Removal Of Obstru...|4.322222222| 

In [8]:
case.select(case.case_closed).toPandas().value_counts()

case_closed
YES            823594
NO              18110
dtype: int64

In [9]:
open_case = case.where((case.case_closed == 'NO'))
open_case.select(min(case.num_days_late)).show()

TypeError: Column is not iterable

2. How many Stray Animal cases are there?


3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?


4. Convert the council_district column to a string column.


5. Extract the year from the case_closed_date column.


6. Convert num_days_late from days to hours in new columns num_hours_late.


7. Join the case data with the source and department data.


8. Are there any cases that do not have a request source?


9. What are the top 10 service request types in terms of number of requests?


10. What are the top 10 service request types in terms of average days late?


11. Does number of days late depend on department?


12. How do number of days late depend on department and request type?